# Text Preprocessing:

Data cleaning is an important and intensive process in Data science which aids in data analysis and building machine learning models.

In [80]:
import pandas as pd
import unidecode 
import re
import time 
import stopwords 
# nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk import word_tokenize
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

import langid
from langdetect import DetectorFactory
from langdetect import detect
from langdetect import detect_langs

import warnings
warnings.filterwarnings('ignore')

In [86]:
def get_data_from_csv(url):
    df = pd.read_csv(url)
    return df

# Now we start preparing the data:

- Lowercase the text
- Remove links
- Remove non english language
- remove punctuation, stop words

https://github.com/kk7nc/Text_Classification

# Stop words
Text and document classification over social media, such as Twitter, Facebook, and so on is usually affected by the noisy nature (abbreviations, irregular forms) of the text corpuses.


# Capitalization
Sentences can contain a mixture of uppercase and lower case letters. Multiple sentences make up a text document. To reduce the problem space, the most common approach is to reduce everything to lower case. This brings all words in a document in same space, but it often changes the meaning of some words, such as "US" to "us" where first one represents the United States of America and second one is a pronoun. To solve this, slang and abbreviation converters can be applied.

# Noise Removal
Another issue of text cleaning as a pre-processing step is noise removal. Text documents generally contains characters like punctuations or special characters and they are not necessary for text mining or classification purposes. Although punctuation is critical to understand the meaning of the sentence, but it can affect the classification algorithms negatively.

In [87]:
def cleaning_preprocessing_data_from_csv(data):  
    cleaned_data = []
    
    for text in data:

        # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
        formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

        # Removing all the occurrences of links that starts with https
        formatted_text = re.sub(r'http\S+', '', formatted_text)

        # Remove all the occurrences of text that ends with .com
        formatted_text = re.sub(r"\ [A-Za-z]*\.com", " ", formatted_text)

        # Remove all whitespaces
        pattern = re.compile(r'\s+') 
        formatted_text = re.sub(pattern, ' ', formatted_text)
        formatted_text = formatted_text.replace('?', ' ? ').replace(')', ') ')

        # Remove accented characters from text using unidecode.
        # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
        remove_character = unidecode.unidecode(formatted_text)

        # Convert text to lower case
        lower_text = remove_character.lower()

        # Pattern matching for all case alphabets
        Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)

        # Limiting all the  repeatation to two characters.
        formatted_text = Pattern_alpha.sub(r"\1\1", lower_text) 

        # Pattern matching for all the punctuations that can occur
        Pattern_Punct = re.compile(r'(\'[.,/#!"$<>@[]^&%^&*?;:{}=_`~()+-])\1{1,}')

        # Limiting punctuations in previously formatted string to only one.
        Combined_Formatted = Pattern_Punct.sub(r'\1', formatted_text)

        # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
        Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)

        # The formatted text after removing not necessary punctuations.
        Formatted_Text = re.sub(r"[^a-zA-Z]+", ' ', Final_Formatted) 

        # Text without stopwords
        remove_stop_words = repr(Formatted_Text)
        stoplist = stopwords.words('english') 

        # Append words to Medium.com
        stoplist.extend(['ago', 'followers', 'pinned', 'read', 'min', 'published', 'days', 'hours', 'the'])
            
        No_StopWords = [word for word in word_tokenize(remove_stop_words) if word.lower() not in stoplist ]

        # Convert list of tokens_without_stopwords to String type.
        words_string = No_StopWords[0]
        words_string = ' '.join(No_StopWords[1:]) 
        
        # Remove more stop words  
        final_text = remove_stopwords(words_string) 
        
        # Split the "'" from the edges
        cleaned_data.append(final_text[:len(final_text)-1])
        
    return cleaned_data

In [88]:
def remove_non_english_articles_and_remove_duplicated_rows(df):
    # dropping ALL duplicate values (keep only one)
    df.drop_duplicates('content', inplace = True)
    
    for topic in df["topic"]:   
        DetectorFactory.seed = 0
        if detect(topic) != "en":
            print("Found different language: " + detect(topic))
            df.drop(df.index[(df["topic"] == topic)], axis=0, inplace=True)
            
    for content in df["content"]:
        DetectorFactory.seed = 0
        if detect(content) != "en":
            print("Found different language: " + detect(content))
            df.drop(df.index[(df["content"] == content)], axis=0, inplace=True) #axis 0 for rows

    return df

In [89]:
def drop_rows_with_short_content(df):
    # also drop nulls
    # split values by whitespace and drop data lt a word
    df = df[df["topic"].str.split().str.len() > 1]
    print(str(df.shape))
    # split values by whitespace and drop data lt a word and bt 5
    df = df[df["category"].str.split().str.len() < 5]
    print(str(df.shape))
    df = df[df["category"].str.split().str.len() > 0]
    print(str(df.shape))
    # split values by whitespace and drop data lt 3 words 
    df = df[df["content"].str.split().str.len() > 3]
    print(str(df.shape))
    return df

## Keep the 'cleaned' data frame

In [90]:
def insert_df_to_csv(topic, category, content, csv_file_name):
    new_df = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    new_df.to_csv(csv_file_name)
    return new_df

In [91]:
df = get_data_from_csv("merge.csv")
print("Before cleaning data the shape of data frame is : " + str(df.shape))
df

Before cleaning data the shape of data frame is : (43954, 1218)


,Unnamed: 0,topic,category,content,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1208,Unnamed: 1209,Unnamed: 1210,Unnamed: 1211,Unnamed: 1212,Unnamed: 1213,Unnamed: 1214,Unnamed: 1215,Unnamed: 1216,Unnamed: 1217
0,7,The Importance the of Auditing Your Attention:...,business,1.3K Followers Pinned One of the most challen...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Even the Best Startup Leaders Have This Weakne...,business,19.1K Followers Pinned All the actionable stu...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Econ Made Me Jack Welch All My Friends,business,32 Followers 16 hours ago Or: How I learned t...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10 Skills That’ll Make You Money (In 2022),business,10.6K Followers Pinned How To Learn Faster By...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Your Biggest Risk Is You,business,647 Followers Published in Towards Data Scien...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43949,42136,10 Steps to Building a Data Collection System ...,technology,268 Followers Published in DataDrivenInvestor...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43950,42137,When Should I Talk to My Child About Periods?,technology,1.99K Followers 23 hours ago Follow up questi...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43951,42138,If I Did Things Differently,technology,23 Followers 22 hours ago Gaming has been wit...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43952,42139,Get Paid Directly by Fans,technology,1.99K Followers Pinned Do at least 7 immediat...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df = drop_rows_with_short_content(df)
print("After drop_rows_with_short_content the shape of data frame is : " + str(df.shape))

df = remove_non_english_articles_and_remove_duplicated_rows(df)
print("After remove_non_english_articles_and_remove_duplicated_rows the shape of data frame is : " + str(df.shape))

title_list = cleaning_preprocessing_data_from_csv(df["topic"])
content_list = cleaning_preprocessing_data_from_csv(df["content"])

new_df = insert_df_to_csv(title_list, df["category"], content_list, "cleaned-scraping-data.csv")
print("After cleaning data the shape of data frame is : " + str(new_df.shape))
new_df

(42946, 1218)
(42817, 1218)
(42817, 1218)
(42575, 1218)
After drop_rows_with_short_content the shape of data frame is : (42575, 1218)
Found different language: de
Found different language: no
Found different language: nl
Found different language: de


KeyboardInterrupt: 